In [1]:
import sys, subprocess, os
import numpy as np
import pandas as pd

In [2]:
all_annotations = {}

for i in range(1, 23):
    all_annotations[i] = pd.read_csv(f"{i}annotationsnew.tsv", sep="\t", header=None)
    all_annotations[i].columns = ["location", "annotation"]

In [3]:
all_annotations[1].head()

,location,annotation
0,69487,missense_variant
1,69569,missense_variant
2,139853,upstream_gene_variant
3,139853,downstream_gene_variant
4,139853,downstream_gene_variant


In [1]:
# Import the significant annotations

sign_annotations = {}

for i in range(1, 23):
    sign_annotations[i] = pd.read_csv(f"{i}signnew.tsv", sep="\t", header=None)
    sign_annotations[i].columns = ["location", "significant"]
    

NameError: name 'pd' is not defined

In [ ]:
# df_comb1 = combined annotations, with annotation entries such as 'splice_region_variant,intron_variant'
# df_comb2 = df_comb1, but with only one annotation per entry
# df_comb = df_comb3 but with unique rows


In [66]:
# Combine the annotations
df_comb1 = {}

for i in range(1, 23):
    df_comb1[i] = pd.merge(all_annotations[i], sign_annotations[i], on="location", how="left")


In [67]:
for i in range(1,23):
    df_comb1[i]['significant'] = df_comb1[i]['significant'].fillna(0)


In [68]:
df_comb1[1].head(6)                                      
                                                                          

,location,annotation,significant
0,69487,missense_variant,0.0
1,69569,missense_variant,0.0
2,139853,upstream_gene_variant,0.0
3,139853,downstream_gene_variant,0.0
4,139853,downstream_gene_variant,0.0
5,139853,"splice_region_variant,intron_variant,non_codin...",0.0


In [69]:
# for the entries in annotation column string we split them by comma, so the 
# multiple entries become: [intron_variant, non_coding_transcript_variant]. 

for i in range(1,23):
    df_comb1[i]['annotation'] = df_comb1[i]['annotation'].str.split(pat=",")

In [72]:
for i in range(1,23):
    print(df_comb1[i].head(6))

   location                                         annotation  significant
0     69487                                 [missense_variant]          0.0
1     69569                                 [missense_variant]          0.0
2    139853                            [upstream_gene_variant]          0.0
3    139853                          [downstream_gene_variant]          0.0
4    139853                          [downstream_gene_variant]          0.0
5    139853  [splice_region_variant, intron_variant, non_co...          0.0
   location            annotation  significant
0     10181  [intergenic_variant]          0.0
1     10574  [intergenic_variant]          0.0
2     10596  [intergenic_variant]          0.0
3     10797  [intergenic_variant]          0.0
4     11247  [intergenic_variant]          0.0
5     11274  [intergenic_variant]          0.0
   location               annotation  significant
0     60197     [intergenic_variant]          0.0
1     60363     [intergenic_variant]   

In [73]:
df_comb2 = {}

for i in range(1,23):
    df_comb2[i] = df_comb1[i].explode('annotation')
    print(df_comb2[i].head(6))

   location               annotation  significant
0     69487         missense_variant          0.0
1     69569         missense_variant          0.0
2    139853    upstream_gene_variant          0.0
3    139853  downstream_gene_variant          0.0
4    139853  downstream_gene_variant          0.0
5    139853    splice_region_variant          0.0
   location          annotation  significant
0     10181  intergenic_variant          0.0
1     10574  intergenic_variant          0.0
2     10596  intergenic_variant          0.0
3     10797  intergenic_variant          0.0
4     11247  intergenic_variant          0.0
5     11274  intergenic_variant          0.0
   location             annotation  significant
0     60197     intergenic_variant          0.0
1     60363     intergenic_variant          0.0
2     60383     intergenic_variant          0.0
3     60596  upstream_gene_variant          0.0
4     60648  upstream_gene_variant          0.0
5     60795  upstream_gene_variant          0.0

In [75]:
df_comb = {}

for i in range(1,23):
    df_comb[i] = df_comb2[i].drop_duplicates()
    print(df_comb[i].head(8))

   location                     annotation  significant
0     69487               missense_variant          0.0
1     69569               missense_variant          0.0
2    139853          upstream_gene_variant          0.0
3    139853        downstream_gene_variant          0.0
5    139853          splice_region_variant          0.0
5    139853                 intron_variant          0.0
5    139853  non_coding_transcript_variant          0.0
8    693731          upstream_gene_variant          0.0
   location          annotation  significant
0     10181  intergenic_variant          0.0
1     10574  intergenic_variant          0.0
2     10596  intergenic_variant          0.0
3     10797  intergenic_variant          0.0
4     11247  intergenic_variant          0.0
5     11274  intergenic_variant          0.0
6     11276  intergenic_variant          0.0
7     11320  intergenic_variant          0.0
   location             annotation  significant
0     60197     intergenic_variant         

   location                     annotation  significant
0    229783                 intron_variant          0.0
0    229783  non_coding_transcript_variant          0.0
1    238434        downstream_gene_variant          0.0
2    240554          upstream_gene_variant          0.0
3    240963          upstream_gene_variant          0.0
4    240983          upstream_gene_variant          0.0
5    244421             intergenic_variant          0.0
6    244426             intergenic_variant          0.0
   location             annotation  significant
0     61098     intergenic_variant          0.0
1     61270     intergenic_variant          0.0
2     61795     intergenic_variant          0.0
3     62731     intergenic_variant          0.0
4     63231  upstream_gene_variant          0.0
5     63244  upstream_gene_variant          0.0
6     63452  upstream_gene_variant          0.0
8     63799  upstream_gene_variant          0.0
   location          annotation  significant
0   9574185  interg

In [90]:
annotations = []
for i in range(1,23):
    annotations.append(df_comb[i]['annotation'].unique().tolist())
    

In [98]:
# get all the annotations we have
from pandas.core.common import flatten
set(list(flatten(annotations)))

{'3_prime_UTR_variant',
 '5_prime_UTR_variant',
 'NMD_transcript_variant',
 'coding_sequence_variant',
 'downstream_gene_variant',
 'incomplete_terminal_codon_variant',
 'intergenic_variant',
 'intron_variant',
 'mature_miRNA_variant',
 'missense_variant',
 'non_coding_transcript_exon_variant',
 'non_coding_transcript_variant',
 'splice_acceptor_variant',
 'splice_donor_variant',
 'splice_region_variant',
 'start_lost',
 'stop_gained',
 'stop_lost',
 'stop_retained_variant',
 'synonymous_variant',
 'upstream_gene_variant'}

In [99]:
# Save df_comb and all annotations as tsv files

for i in range(1,23):
    df_comb[i].to_csv(f"{i}permutationfile.tsv", sep="\t", index=False) 


In [ ]:
import time
import random

In [104]:
# Create dictionaries to store values for each of the counts 

annotations = ['3_prime_UTR_variant','5_prime_UTR_variant','NMD_transcript_variant','coding_sequence_variant','downstream_gene_variant','incomplete_terminal_codon_variant','intergenic_variant','intron_variant','mature_miRNA_variant','missense_variant','non_coding_transcript_exon_variant','non_coding_transcript_variant','splice_acceptor_variant','splice_donor_variant','splice_region_variant','start_lost','stop_gained','stop_lost','stop_retained_variant','synonymous_variant','upstream_gene_variant']

In [107]:
# Now put annotations as column headers 

df_comb[1].head(10)

,location,annotation,significant
0,69487,missense_variant,0.0
1,69569,missense_variant,0.0
2,139853,upstream_gene_variant,0.0
3,139853,downstream_gene_variant,0.0
5,139853,splice_region_variant,0.0
5,139853,intron_variant,0.0
5,139853,non_coding_transcript_variant,0.0
8,693731,upstream_gene_variant,0.0
9,693731,downstream_gene_variant,0.0
11,707522,intron_variant,0.0


In [139]:
df_comb[1].loc[df_comb[1].annotation == '3_prime_UTR_variant']

,location,annotation,significant,3_prime_UTR_variant
2689,879676,3_prime_UTR_variant,0.0,True
2700,879687,3_prime_UTR_variant,0.0,True
2711,879911,3_prime_UTR_variant,0.0,True
3478,898787,3_prime_UTR_variant,0.0,True
3489,899336,3_prime_UTR_variant,0.0,True
...,...,...,...,...
3154743,249151879,3_prime_UTR_variant,0.0,True
3154777,249152001,3_prime_UTR_variant,0.0,True
3155834,249212725,3_prime_UTR_variant,0.0,True
3155839,249212878,3_prime_UTR_variant,0.0,True


In [142]:
df_comb[1]

,location,annotation,significant,3_prime_UTR_variant
0,69487,missense_variant,0.0,False
1,69569,missense_variant,0.0,False
2,139853,upstream_gene_variant,0.0,False
3,139853,downstream_gene_variant,0.0,False
5,139853,splice_region_variant,0.0,False
...,...,...,...,...
3155893,249229734,upstream_gene_variant,0.0,False
3155894,249229967,upstream_gene_variant,0.0,False
3155895,249230279,upstream_gene_variant,0.0,False
3155896,249230793,non_coding_transcript_exon_variant,0.0,False


In [135]:
array1

array([False, False, False, ..., False, False, False])

In [136]:
df_comb[1]['3_prime_UTR_variant'] = array1.tolist()

/tmp/ipykernel_71695/383592306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comb[1]['3_prime_UTR_variant'] = array1.tolist()


In [14]:
time_count = {}
missense_count = {}

for j in range(1, 50):
    count_one = {} # count transcript
    df_perm = {} # the dataframe containing values after 1 permutation
        # Now permute the 1s in the column 'annotationsig' in df_perm[i]
    for i in range(1, 23):
        r = random.randint(0, len(all_annotations[i])) # permutes each chromosome by a different number
            # print(r)
        df_perm[i] = df_comb[i].copy()
        df_perm[i]["annotation"] = np.roll(df_comb[i]["annotation"], r) # permute the significant values
                # count how many rows with entry 'Transcript', ' Regulatoryfeature' and 'Motiffeature' in column 'annotationall' have 1s
        count_one[i] = df_comb[i][(df_comb[i]["significant"] == 1) & (df_comb[i]['annotation'].str.contains(r'missense_variant', na=True))].count()
    missense_count[j] = count_one
    
end = time.time()
time_count[k] = end - start


NameError: name 'start' is not defined

In [14]:
# missense_count is actually counting the non_coding_transcript_variants
time_count.values()

dict_values([238.59135150909424, 237.02543330192566, 236.17738914489746, 236.55043244361877, 235.92396092414856, 237.98740410804749, 242.7115273475647, 243.05419611930847, 245.37424325942993, 246.08268308639526, 246.8887312412262, 246.70415329933167, 246.90488004684448, 246.60206246376038, 246.33857345581055, 246.22503209114075, 245.91034746170044, 246.3363447189331, 246.13405442237854, 245.62321949005127, 245.86300015449524, 246.71079802513123, 245.44843816757202, 246.29271912574768, 246.64891028404236, 246.37998938560486, 246.44418025016785, 246.4673867225647, 246.2163896560669, 245.74071550369263, 246.423259973526, 246.09086918830872, 246.08154606819153, 246.57220673561096, 246.3519959449768, 246.31402802467346, 246.74247550964355, 246.47401309013367])